Laboratorio 1 - Regresión lineal 
Integrantes: 
Silvana Echeverry - 202310470
David Mora - 
Carlos Vargas

In [2]:
#importando librerias 

import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import numpy as np
import seaborn as sns

In [3]:
data_train = pd.read_csv('data/datos_entrenamiento_laboratorio1(train_data).csv')
data_test = pd.read_csv('data/datos_validacion_laboratorio1(test_data).csv')

1. Entendimiento de los datos 

Para esta primera etapa, decidimos hacer uso de diferentes funciones para entender como se comportaban nuestros datos, como: 
* Head() -> muestra las primera filas del data frame con las columnas 
* isna().sum() -> nos permitio contar los datos que tenian valores nulos,
* dtypes -> saber los tipos de las variables que tenemos

Despues de hacer esta exploración de los datos, nos dimos cuenta que ciertas variables importantes estaban con valores nulos, o variables importantes estaban con valores que no les correspondian, por ejemplo: 
- edad: object -> y deberia ser int
- market value: object -> y deberia ser int
- dia de partido: object -> fecha 
y con los valores nulos: 
- Edad 48
- Nacionalidad 32
- %_de_regates_exitosos 28.722
- %_de_duelos_aereos_ganados 20.957
- market_value 12.894 (solo esta en el archivo de entrenamiento)
- contract_date 12.916

nota: el pais también deberia cambiarse para dejarlo solamente con su codigo ejm: ARG -> Argentina

Nuestra variable objetivo es predecir el valor de mercado de un jugador (market_value) a partir de sus estadísticas registradas durante partidos. 

Cada fila del data set representa el desempeño de un jugador en un partido especifico (fecha de partido). Nos podemos dar cuenta que hay multiples registros de cada jugador en cada uno de los partidos. 

Además, encontramos que ciertas columnas estan repetidas con datos duplicados como: 
- xAG
- Pasos_Progresivos
- Regates_Exitosos
- Pases_completados

Para hacer una correcta estimación, en esta etapa consideramos que hacen falta variables en cuenta para la estimación, como: 
- partidos_jugados: Cantidad total de partidos registrados por jugador en la temporada
- valor_por_partido: Valor de mercado dividido entre partidos_jugados, ya que segun el partido pueden tener diferente valor
- posicion_general: una variable que categorice la posición del jugador
- años_fincontrato: calcular los años hasta que se le acabe el contrato

Variables mas significativas para nuestro modelo (hasta el momento), consideramos que en esta primera revisión estos son los datos mas relevantes para nuestro modelo 
1. Edad
2. posicion_general (nueva variable)
3. partidos_jugados (nueva variable)
4. Goles
5. xG
6. xA
7. xAG
8. Acciones_que_crean_tiros
9. Pases_al_hueco
10. Pases_en_ultimo_tercio
11. Regates_exitosos
12. Toques_en_area_rival
15. Faltas_cometidas
16. Valla_no_vencida 
17. valor_por_partido (nueva variable)
18. años_fincontrato (nueva variable)
19. Pases_progresivos
20. Pases_intentados
21. Nacionalidad 

In [9]:
' Usamos estas lineas de codigo para explorar los datos '
#data_test.head()
#data_train.head()
data_train.isna().sum() #verificando datos nulos
#data_test.isna().sum()
#data_test.dtypes
#data_train.dtypes

Jugador                             0
Nacionalidad                        2
Posicion                            0
Edad                                0
Dia_partido                         0
Goles                               0
Tiros Totales                       0
xG                                  0
npxG                                0
xAG                                 0
xAG.1                               0
Acciones_que_crean_tiros            0
Pases_intentados                    0
Pases_intentados.1                  0
Pases_intentados.2                  0
Pases_progresivos                   0
Pases_progresivos.1                 0
Regates_exitosos                    0
Regates_exitosos.1                  0
Pases_medios_completados            0
Pases_largos_completados            0
xAG.2                               0
xAG.3                               0
xA                                  0
Pases_en_ultimo_tercio              0
Pases_balon_vivo                    0
Pases_balon_

2. Preparación de los datos

Esta etapa lo dividimos en: 
1. Eliminar columnas duplicadas
2. Corregir tipos de datos
3. Tratar valores nulos
4. Normalizar categorías
5. Crear nuevas variables
6. Codificar variables categóricas (one hot encoding)
7. Filtrar registros incompletos
8. Verificar multicolinealidad
9. Separar variables predictoras y variable objetivo

limpiar, fill y luego filtro 

In [5]:
#convertir la fecha de contrato a datetime
data_train['contract_date'] = pd.to_datetime(data_train['contract_date'], errors='coerce')

#calculo el promedio de los contratos de los jugadores para rellenar los nulos, lo hice para revisar cuanto en promedio dura un contrato de un jugador y añadirle esa fecha 
contract_date_mean = data_train['contract_date'].dropna().mean()

data_train['contract_date'] = data_train['contract_date'].fillna(contract_date_mean)

# aca saco la fecha maxima de contrato por jugador para quedarme con la ultima fecha de contrato de cada jugador 
idxl = data_train.groupby('Jugador')['contract_date'].idxmax()

# ahora con esto filtro el data frame para quedarme con la ultima fecha de contrato de cada jugador
data_train = data_train.loc[idxl]

data_train.isna().sum()

/var/folders/0g/xm2gc5sx67sds29_nv2g25n00000gn/T/ipykernel_79573/1429246051.py:2: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_train['contract_date'] = pd.to_datetime(data_train['contract_date'], errors='coerce')


Jugador                              0
Nacionalidad                         6
Posicion                             0
Edad                                 6
Dia_partido                          0
Goles                                0
Tiros Totales                        0
xG                                   0
npxG                                 0
xAG                                  0
xAG.1                                0
Acciones_que_crean_tiros             0
Pases_intentados                     0
Pases_intentados.1                   0
Pases_intentados.2                   0
Pases_progresivos                    0
Pases_progresivos.1                  0
Regates_exitosos                     0
Regates_exitosos.1                   0
Pases_medios_completados             0
Pases_largos_completados             0
xAG.2                                0
xAG.3                                0
xA                                   0
Pases_en_ultimo_tercio               0
Pases_balon_vivo         

In [6]:

# 1. Convertir a string
data_train["Edad"] = data_train["Edad"].astype(str)

# 2. Extraer la parte antes del guión
data_train["Edad"] = data_train["Edad"].str.split('-').str[0]

# 3. Reconvertir 'nan' como texto a NaN real
data_train["Edad"] = data_train["Edad"].replace("nan", np.nan)

# 4. Rellenar los NaN con 0 o la mediana si prefieres
data_train["Edad"] = data_train["Edad"].fillna(0)

# 5. Convertir a int
data_train["Edad"] = data_train["Edad"].astype(int)

In [7]:

#tomar la ultima edad registrado 
#eliminar los ultimos digitos 
#dia de partido en el formato datetime
data_train['Dia_partido'] = pd.to_datetime(data_train['Dia_partido'], errors='coerce')

# 1. Eliminar registros donde 'Dia_partido' es NaN
data_train_valid_fecha = data_train.dropna(subset=['Dia_partido'])

# 2. Obtener el índice del último partido por jugador
idx_fecha = data_train_valid_fecha.groupby('Jugador')['Dia_partido'].idxmax()

# 3. Filtrar solo esos registros
data_train = data_train_valid_fecha.loc[idx_fecha]

# 4. Verificar valores nulos
data_train.isna().sum()

Jugador                              0
Nacionalidad                         6
Posicion                             0
Edad                                 0
Dia_partido                          0
Goles                                0
Tiros Totales                        0
xG                                   0
npxG                                 0
xAG                                  0
xAG.1                                0
Acciones_que_crean_tiros             0
Pases_intentados                     0
Pases_intentados.1                   0
Pases_intentados.2                   0
Pases_progresivos                    0
Pases_progresivos.1                  0
Regates_exitosos                     0
Regates_exitosos.1                   0
Pases_medios_completados             0
Pases_largos_completados             0
xAG.2                                0
xAG.3                                0
xA                                   0
Pases_en_ultimo_tercio               0
Pases_balon_vivo         

In [8]:


data_train['market_value_f'] = data_train['market_value'].str.replace('-', '0')
data_train['market_value_f'] = data_train['market_value_f'].str.replace('error', '0')
data_train['market_value_f'] = data_train['market_value_f'].str.replace('€', '')
data_train['market_value_ultimodigito'] = data_train['market_value_f'].str[-1] #ultimo digito
data_train['market_value_ultimodigito'] = data_train['market_value_ultimodigito'].map({'m': 10000000, 'k': 1000})
data_train['market_value_f'] = data_train['market_value_f'].str.replace('k', '').str.replace('m', '').astype(float)
data_train['market_value_f'] = data_train['market_value_f'] * data_train['market_value_ultimodigito']


data_train = data_train.dropna(subset=['market_value'])

#data_train[data_train['market_value_f'].isna()] #verficaa con 1 y 0 si esta nula, filtra las filas de ese valor en null 

,Jugador,Nacionalidad,Posicion,Edad,Dia_partido,Goles,Tiros Totales,xG,npxG,xAG,...,Pases_recibidos,Pases_progresivos_recibidos,Faltas_cometidas,Centros,Duelos_aereos_ganados,%_de_duelos_aereos_ganados,market_value,contract_date,market_value_f,market_value_ultimodigito
37745,Abdoulaye Bamba,ci CIV,CB,34,2024-08-18,0,0,0.0,0.0,0.1,...,8,0,0,1,2,66.7,-,2029-06-30 00:00:00.000000000,NaN,NaN
37938,Benjamin Andre,fr FRA,CM,34,2024-08-24,0,1,0.0,0.0,0.0,...,71,3,0,1,1,50.0,-,2031-06-30 00:00:00.000000000,NaN,NaN
46906,Lanroy Machine,fr FRA,RW,19,2025-05-10,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,-,2028-04-05 09:41:01.780937216,NaN,NaN
24871,Matteo Pessina,it ITA,CM,27,2024-08-17,0,0,0.0,0.0,0.0,...,6,0,0,0,0,NaN,-,2028-04-05 09:41:01.780937216,NaN,NaN
564,Mykhailo Mudryk,ua UKR,LW,23,2024-08-25,0,0,0.0,0.0,0.0,...,10,3,1,1,0,0.0,-,2031-06-30 00:00:00.000000000,NaN,NaN
12403,Victor Meseguer,es ESP,CM,25,2024-08-19,689,0,0.0,0.0,0.0,...,7,1,0,0,1,50.0,error,2028-04-05 09:41:01.780937216,NaN,NaN


In [ ]:
#añadir un mini indice para luego ver que borrar 

#juntar para tener mas globales las posiciones, cambiar valor dependiendo si esta en una lista que defini 